# All Courses From CourseExplorer

**GET ALL DEPARTMENT NAME**

In [2]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd

year_term = "2022/spring"
school_url = "https://courses.illinois.edu/schedule/"+(year_term)

response = requests.get(school_url)

department_name = []

page = soup(response.text, 'html.parser')

for link in page.find_all('a'):
  candidates = str(link.get('href'))
  if (candidates.find('/schedule/2022/spring/') != -1):
    department_name.append(candidates[22:])

**GET ALL COURSE NUMBER & NAME**

In [3]:
import multiprocessing
from multiprocessing import Pool
import sys

def job(diction, url, dep):
  response_coures_list = requests.get(url)
  page_courses_list = soup(response_coures_list.text, 'html.parser')
  courses_name_nums = dict()
  for st_a in page_courses_list.find_all('a'):
    nums_pre = str(st_a.get('href'))
    if ((year_term + '/' + dep) in nums_pre and 'pdf' not in nums_pre):
      num = str(st_a.get('href'))[-3:]
      name = st_a.string
      courses_name_nums[num] = name

  diction[dep] = courses_name_nums
  


url_list=[]
manager = multiprocessing.Manager()
all_data = manager.dict()

for dep in department_name:
  dep_url = school_url + '/' + dep
  url_list.append(dep_url)

p = Pool(15)
sys.setrecursionlimit(2500000)
for i in range(len(url_list)):
  p.apply_async(job, args=(all_data, url_list[i], department_name[i]))
  
p.close()
p.join()

**SAVE NAME & NUMBER**

In [ ]:
df = pd.DataFrame()
for i in all_data.items():
  temp_list = []
  for j in i[1].keys():
    temp_list.append(j + " " +i[1][j])
  temp = pd.DataFrame({i[0]: temp_list})
  df = pd.concat([df, temp], axis=1)

df.to_csv("course_num_name.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



MessageError: ignored

**SAVE CS Department Information**

In [ ]:
#Multiprocessing
import re
import json
import os
if not (os.path.exists('course_info')):
  os.mkdir('course_info')
#school_url = "https://courses.illinois.edu/schedule/"+(year_term)

def job_2(department):
  #print("here")
  department_dataframe = pd.DataFrame()
  for course_number in all_data[department]:
    course_url = school_url + '/' + department + '/' + course_number
    #print(course_url)
    response = requests.get(course_url)
    text = response.text
    page = soup(text, 'html.parser')
    pattern = re.compile(r"var sectionDataObj = (.*?);$", re.MULTILINE | re.DOTALL) #get target variable using regular expression
    script = page.find("script", text=pattern)
    info = pattern.search(script.prettify()).group(1)
    data = json.loads(info)     #data: list of dictionary, where each dictionary is a course
    #print(data[0]["crn"])
    crn = []
    type_ = []
    section = []
    time = []
    day = []
    location = []
    instructor = []
    for course in data:
      crn.append(course["crn"].strip())
      type_.append(course["type"].split("meeting\">")[1].split("</div>")[0].strip())
      section.append(course["section"].split("meeting\">")[1].split("</div>")[0].strip())
      time.append(course["time"].split("meeting\">")[1].split("</div>")[0].strip())
      day.append(course['day'].split("meeting\">")[1].split("</div>")[0].strip())
      location.append(course['location'].split("meeting\">")[1].split("</div>")[0].strip())
      instr_info = course['instructor'].split("meeting\">")[1].split("</div>")[0].strip()
      if instr_info:
        instr_info = instr_info[:-6].replace("<br />", " & ", 10)
      instructor.append(instr_info)
    course_data = pd.DataFrame({"Course":department+course_number, "Name":all_data[department][course_number].strip(), "CRN":crn, "Type":type_, "Section":section, "Time":time, "Day":day, "Location":location, "Instructor":instructor})
    department_dataframe = pd.concat([department_dataframe, course_data], ignore_index=True)
  department_dataframe.to_csv('course_info/' + department + '.csv')

p = Pool(16)
sys.setrecursionlimit(2500000)
for i in range(len(department_name)):
#for i in range(1):
  p.apply_async(job_2, (department_name[i],))  
p.close()
p.join()

!zip -r /content/course_info.zip /content/course_info

  adding: content/course_info/ (stored 0%)
  adding: content/course_info/CMN.csv (deflated 80%)
  adding: content/course_info/CW.csv (deflated 73%)
  adding: content/course_info/FIN.csv (deflated 80%)
  adding: content/course_info/ACES.csv (deflated 57%)
  adding: content/course_info/TE.csv (deflated 69%)
  adding: content/course_info/HEBR.csv (deflated 48%)
  adding: content/course_info/AAS.csv (deflated 70%)
  adding: content/course_info/ARCH.csv (deflated 77%)
  adding: content/course_info/CZCH.csv (deflated 10%)
  adding: content/course_info/VM.csv (deflated 68%)
  adding: content/course_info/ANSC.csv (deflated 76%)
  adding: content/course_info/AGCM.csv (deflated 52%)
  adding: content/course_info/ARTJ.csv (deflated 56%)
  adding: content/course_info/GRK.csv (deflated 61%)
  adding: content/course_info/GLBL.csv (deflated 67%)
  adding: content/course_info/EDPR.csv (deflated 82%)
  adding: content/course_info/FAA.csv (deflated 51%)
  adding: content/course_info/HIST.csv (deflated 7